## Fuentes de datos

**Directorio:** `\\srvapwb12\SCG_INFORMACION_EXTERNA$\USUARIOS\GCIA BANCA DIGITAL` (Ana Belén Juncos)

**Contenido de la carpeta:**

- Usuarios de platadorma digital `ORA_Users.txt`
- Crosseling de productos de Febrero a Abril `CLIPROD_CROSSELLING_YYYYMM.txt`
- Transacciones monetarias y no monetares de Febrero a Abril: `Detalle_Trx_Monetarias_YYYYMM.del` / `NoMonetarias_YYYYMM.del`

**Directorio:** `\\srvapwb12\SCG_INFORMACION_EXTERNA$\QV\USR\OUTPUT\Gestar` (Marcela Terreno)

**Contenido de la carpeta:**  

- Transacciones mensuales de GESTAR - `Transacciones_Gestar_YYYYMM_NVO.txt`

## Declaraciones generales

In [1]:
# Importo las librerias necesarias
import pandas as pd
import datetime as dt
import numpy as np
import os

In [2]:
# Funciones globales de uso general
def dateparse_(x):
    try:
        date_ = dt.datetime.strptime(x, '%d/%m/%Y')
    except:
        date_ = dt.datetime(1900, 1, 1)
    return date_
dateparse = lambda x: dateparse_(x)

## Depuración y generación del archivo de usuarios

### Estructura del archivo

|Campo    |Descripción   |
|---    |---   |
|ID_USER    |Id de usuario (Hash de 32 bytes)   |
|EMAIL  |Correo electronico   |
|FIRST_NAME |Nombre   |
|LAST_NAME  |Apellido   |
|MOBILE_NUMBER  |Teléfono celular   |
|DOCUMENTO  |Documento (CUIT/CUIL)   |
|FechaCreacion  |Fecha de OnBoarding   |
|LAST_LOGIN |Último logín en la plataforma digital BANCON   |
|SegFactor  |Segundo factor de autenticación   |

In [3]:
# Lectura de archivo
file = r'\\srvapwb12\SCG_INFORMACION_EXTERNA$\USUARIOS\GCIA BANCA DIGITAL\ORA_Users.txt'
df_us = pd.read_csv(file, 
    delimiter=';',
    encoding='latin-1',
    dtype={"DOCUMENTO": str, "LAST_LOGIN": str, "SegFactor": str},
    parse_dates=["FechaCreacion", "LAST_LOGIN"],
    date_parser=dateparse,
    usecols=["ID_USER", "DOCUMENTO", "FechaCreacion", "LAST_LOGIN", "SegFactor"])
df_us.shape

(917541, 5)

In [5]:
# Elimino los registros con fecha de último logín 1900-01-01
df_us = df_us[df_us["LAST_LOGIN"] != dt.datetime(1900, 1, 1)]

In [6]:
# Averiguar si hay o no CUIT/CUILs duplicados
ids = df_us["DOCUMENTO"]
df_us[ids.isin(ids[ids.duplicated()])].sort_values("DOCUMENTO")

,ID_USER,DOCUMENTO,FechaCreacion,LAST_LOGIN,SegFactor
484554,ec0258dd773d4e518c261ca4a1291a0e,20043106261,2020-12-20,2021-09-10,SMS
584347,a40b6b59c911450b892700b8a7549b0a,20043106261,2021-11-09,2022-06-03,Softoken
128921,ca859587dc2f4368a0a91e2e583d34be,20046189958,2019-09-09,2019-09-12,SMS
114931,a8e526c2f12d40dcaf6b733df6f0fbf3,20046189958,2019-08-21,2022-05-31,SMS
479361,79a200d4ad454a63a25c61cc492ab9b7,20049293209,2021-03-02,2022-06-03,Softoken
...,...,...,...,...,...
898028,bbfdf324ce194746968c10eb15795361,27960094676,2020-02-13,2021-03-28,SMS
235572,a5c980c815654b3fadb0e58ec0458840,27960159832,2020-02-28,2021-03-18,SMS
814383,a841dbca361a4a4f9e7622c3171ecc46,27960159832,2021-04-06,2022-06-02,Softoken
537490,c9d18053509a4af1bcd52e6df537edc7,30710898967,2021-08-11,2021-08-12,No tiene segundo factor


In [7]:
# Hay duplicados. Me quedo con el registro con la fecha de último login mas reciente
df_us = df_us.sort_values("LAST_LOGIN").groupby("DOCUMENTO").tail(1)
df_us.shape

(903291, 5)

In [8]:
# Grabo
file = os.path.abspath('..\Export') + r'\usuarios.csv'
df_us.to_csv(file, index=False)

In [9]:
# Borro lo que ya no es necesario para econnomizar memoria
df_us.drop(["FechaCreacion"], axis=1, inplace=True)

## Depuración y generación del archivo de crosseling

### Estructura del archivo

In [9]:
t_cs = {
    "CUITCLIE": str,
    "RELACION": str,
    "TIPOIMPN": str,
    "NATUJURI": str,
    "SUCURSAL": str,
    "BCRASECT": str,
    "BCRASEGO": str,
    "GRUPO_CLIENTE": str,
    "GRUPOGENERAL": str,
    "SUBGRUPO": str,
    "SEGMENTO_SUELDOS": str,
    "SEGMENTO_BANCAS": str
}

uc_cs = [
    "CUITCLIE",
    "RELACION",
    "TIPOIMPN",
    "NATUJURI",
    "SUCURSAL",
    "BCRASECT",
    "BCRASEGO",
    "GRUPO_CLIENTE",
    "GRUPOGENERAL",
    "SUBGRUPO",
    "SEGMENTO_SUELDOS",
    "SEGMENTO_BANCAS",
    "FUM",
    "ES_CAPITA",
    "CL_ACTIVO_FUM",
    "TIENE_I_CA_D",
    "TIENE_I_CA_P",
    "TIENE_I_CBESA",
    "TIENE_I_CC",
    "TIENE_I_CCAD",
    "TIENE_I_CS",
    "TIENE_I_FCI",
    "TIENE_I_PF",
    "TIENE_I_PR_HIP",
    "TIENE_I_PR_PERS",
    "TIENE_I_PREP",
    "TIENE_I_SE_PATR",
    "TIENE_I_SEG_VIDA",
    "TIENE_I_VISA",
    "TIENE_I_TJ_SOC",
    "SALDO_COMPUTO_SIST_2",
    "SALDO_COMPUTO_SIST_CC_MB",
    "SALDO_COMPUTO_SIST_CA_P",
    "SALDO_COMPUTO_SIST_CC_P",
    "SALDO_COMPUTO_SIST_FCI",
    "CROSS_IND"
]

In [18]:
# Lectura de archivo
# Solo tomamos Abril porque en la reunión de Data Driven se concluyó que solo el último es relevante
file = r'\\srvapwb12\SCG_INFORMACION_EXTERNA$\USUARIOS\GCIA BANCA DIGITAL\CLIPROD_CROSSELLING_202204.txt'
df_cs = pd.read_csv(file, 
    delimiter='|',
    nrows=5,
    encoding='latin-1',
    dtype=t_cs,
    parse_dates=["FUM"],
    usecols=uc_cs)
df_cs.shape

(5, 37)

In [13]:
# Anonimizo la identificación del usuario tomando el hash de la tabla de usuarios
df_cs = pd.merge(df_us, df_cs, how='inner', left_on='DOCUMENTO', right_on='CUITCLIE')
df_cs.drop(["DOCUMENTO", "CUITCLIE"], axis=1, inplace=True)

In [14]:
# Reemplazo los valores SI/NO y la convierto en una variable dicotómica
df_cs["CL_ACTIVO_FUM"].replace({"SI": 1, "NO": 0}, inplace=True)

In [12]:
# Grabo
file = os.path.abspath('..\Export') + r'\crosseling.csv'
df_cs.to_csv(file, index=False)

In [11]:
# Libero memoria
del df_cs
df_us.drop(["LAST_LOGIN", "SegFactor"], axis=1, inplace=True)

## Depuración y generación del archivo de reclamos

### Estructura del archivo

|Campo    |Descripción   |
|---    |---   |
|Periodo    |Perído   |
|ID |Id Gestar   |
|CASETYPEID |Id de tipo caso   |
|CASETYPE   |Descripción de tipo de caso   |
|CANORIGID  |Id de canal originante   |
|CANORIG    |Descripción de canal originante   |
|TYPE_CODSUCURSAL   |Sucursal   |
|STATEID    |Id de estado   |
|State  |Descripción de estado   |
|CREATEDDATE    |Fecha de creación del caso YYYYMMDD   |
|CREATEDMONTH   |**Excluir** Mes de creación del caso   |
|ENDDATE    |Fecha de cierre del caso   |
|SING_TIPODOC   |Tipo de documento   |
|SING_NRODOC    |Número de documento **(Ojo! A veces cargan acá el CUIT/CUIL y dejan el campo CUIT vacío)**  |
|CUIT   |CUIT   |
|CLIENTE    |**Excluir** Nombre del cliente   |
|TYPE_NROCTA    |Tipo de cuenta   |
|TYPE_NROTARJETA    |Número de cuenta   |
|NIVEL 1    |Clasificación del caso Nivel 1   |
|NIVEL 2    |Clasificación del caso Nivel 2   |
|NIVEL 3    |Clasificación del caso Nivel 3   |
|SOLVER |**Excluir** Nombre del Resolutor   |
|TYPE_MARCATARJETA  |Tipo de marca de tarjeta   |
|TPOSOLUCION_CALCULADO  |Tipo de solución   |
|TYPE_IMPORTE_TOTAL_RECLAMADO   |Importe total reclamado   |
|TYPE_IMPORTE_TOTAL_REINTEGRADO |Importe total reintegrado   |
|Motivo_Baja_Producto   |Motivo de baja del producto   |
|VIA    |**Via?**  |
|CREADO POR |**Excluir** Nombre de quien cargó el caso   |
|ID CREADOR |**Excluir** Id del creador del caso   |
|ACEPTA CONTACTO    |Acepta contacto SI/NO   |
|EMAIL  |**Excluir** Correo electrónico   |
|TYPE MOTIVO GESTION BANCON |**Motivo de gestión BANCON**   |
|DIAS_DEMORA_DETALLE    |Detalle días de demora   |
|CASETYPEPATH   |**Ruta de tipo de caso?**   |
|GRUPORESPONSABLE   |**Excluir** Grupo responsable   |
|STARTDATE  |**Fecha y hora de carga del caso?**   |
|CUSTOMERID |**Excluir** Id de cliente   |
|SING_FECHANACIMIENTO   |**SING?** **Excluir** Fecha de nacimiento   |
|SING_SEXO  |**SING?** **Excluir** Sexo   |
|SING_PROVINCIA |**SING?** **Excluir** Provincia   |
|SING_BARRIO    |**SING?** **Excluir** Barrio   |
|CUSTWORKCITY   |**Excluir** Ciudad laboural del cliente   |
|TYPE_SUCURSAL  |**Excluir** Sucursal   |
|GRUPORESPONSABLEID |**Excluir** Id de grupo resolutor   |
|TYPE_TC_TIPOTARJETA    |**TC** Tipo de tarjeta   |
|TYPE_TARJETA_NRO   |Número de tarjeta   |
|TYPE_RUBRO_COMERCIO    |Rubro del comercio   |
|type_importe_total_reclamado_dolares   |Importe total reclamado en dólares   |
|TYPE_CONVENIO  |Convenio   |
|TYPE_UBICACION |Ubicación   |
|type_tipo_plataforma_bancon    |Tipo de plataforma BANCON   |
|type_motivo_gestion_bancon_empresas    |Motivo de gestión BANCON empresas   |
|type_motivo_consulta_bancon_empresas   |Motivo consulta BANCON empresas   |
|type_motivo_consulta_bancon    |Motivo consulta BANCON   |


In [229]:
uc_gt = [
    "ID",
    "CASETYPEID",
    "CASETYPE",
    "CANORIGID",
    "CANORIG",
    "TYPE_CODSUCURSAL",
    "STATEID",
    "State",
    "CREATEDDATE",
    "CREATEDMONTH",
    "ENDDATE",
    "SING_TIPODOC",
    "SING_NRODOC",
    "CUIT",
    "TYPE_NROCTA",
    "TYPE_NROTARJETA",
    "NIVEL 1",
    "NIVEL 2",
    "NIVEL 3",
    "TYPE_MARCATARJETA",
    "TPOSOLUCION_CALCULADO",
    "TYPE_IMPORTE_TOTAL_RECLAMADO",
    "TYPE_IMPORTE_TOTAL_REINTEGRADO",
    "Motivo_Baja_Producto",
    "VIA",
    "ACEPTA CONTACTO",
    "TYPE MOTIVO GESTION BANCON",
    "DIAS_DEMORA_DETALLE",
    "CASETYPEPATH",
    "STARTDATE",
    "TYPE_TC_TIPOTARJETA",
    "TYPE_TARJETA_NRO",
    "TYPE_RUBRO_COMERCIO",
    "type_importe_total_reclamado_dolares",
    "TYPE_CONVENIO",
    "TYPE_UBICACION",
    "type_tipo_plataforma_bancon",
    "type_motivo_gestion_bancon_empresas",
    "type_motivo_consulta_bancon_empresas",
    "type_motivo_consulta_bancon"
]

t_gt = {
    "ID": str,
    "CASETYPEID": str,
    "CANORIGID": str,
    "TYPE_CODSUCURSAL": str,
    "STATEID": str,
    "CREATEDMONTH": str,
    "SING_TIPODOC": str,
    "SING_NRODOC": str,
    "CUIT": str,
    "TYPE_NROCTA": str,
    "TYPE_NROTARJETA": str,
    "TYPE_MARCATARJETA": str,
    "TPOSOLUCION_CALCULADO": str,
    "TYPE_IMPORTE_TOTAL_RECLAMADO": str,
    "TYPE_IMPORTE_TOTAL_REINTEGRADO": str,
    "Motivo_Baja_Producto": str,
    "VIA": str,
    "TYPE_TARJETA_NRO": str,
    "TYPE_RUBRO_COMERCIO": str,
    "type_importe_total_reclamado_dolares": str,
    "TYPE_CONVENIO": str,
    "TYPE_UBICACION": str,
    "type_tipo_plataforma_bancon": str,
    "type_motivo_gestion_bancon_empresas": str,
    "type_motivo_consulta_bancon_empresas": str,
    "type_motivo_consulta_bancon": str
}

In [280]:
# Lectura de los archivos
m_ = "w"
for i in ["202202", "202203", "202204"]:

    file = r'\\srvapwb12\SCG_INFORMACION_EXTERNA$\QV\USR\OUTPUT\Gestar\Transacciones_Gestar_{}_NVO.txt'.format(i)
    df_gt = pd.read_csv(file, 
        delimiter='|',
        encoding='latin-1',
        dtype=t_gt,
        parse_dates=["CREATEDDATE", "ENDDATE", "STARTDATE"],
        usecols=uc_gt)

    # Donde haya un CUIT con valor nulo lo plancho con el valor del DNI si su longitud de >= 11
    df_gt["CUIT"] = np.where(pd.isnull(df_gt["CUIT"]) & (df_gt["SING_NRODOC"].str.len()>=11), df_gt["SING_NRODOC"], df_gt["CUIT"])
    # Borrar todas las filas sin CUIT
    df_gt = df_gt[~pd.isnull(df_gt["CUIT"])]
    # Corrijo todos los CUIT que tienen tres ceros adelante
    df_gt.loc[(df_gt["CUIT"].str.len()>11) & (df_gt["CUIT"].str[0:3] == "000"), "CUIT"] = df_gt["CUIT"].str[3:14]
    # Corto los CUIT restantes a 11 posiciones (muchos tienen una coma detras)
    df_gt.loc[(df_gt["CUIT"].str.len()>11), "CUIT"] = df_gt["CUIT"].str[0:11]
    # Le saco el guión al mes
    df_gt["CREATEDMONTH"] = df_gt["CREATEDMONTH"].str[:4] + df_gt["CREATEDMONTH"].str[5:7]
    
    # Anonimizo la identificación del usuario tomando el hash de la tabla de usuarios
    df_gt = pd.merge(df_us, df_gt, how='inner', left_on='DOCUMENTO', right_on='CUIT')
    df_gt.drop(["DOCUMENTO", "CUIT"], axis=1, inplace=True)

    # Grabo
    file = os.path.abspath('..\Export') + r'\reclamos.csv'
    df_gt.to_csv(file, index=False, mode=m_, header=(m_ == "w"))
    print(file, 'Filas:', len(df_gt.index), m_, (m_ == "w")) 
    m_ = "a"


c:\Users\e21719832\Desktop\DataDriven\Data\Export\reclamos.csv Filas: 126900 w True
c:\Users\e21719832\Desktop\DataDriven\Data\Export\reclamos.csv Filas: 147237 a False
c:\Users\e21719832\Desktop\DataDriven\Data\Export\reclamos.csv Filas: 132175 a False


In [ ]:
# Libero memoria
del df_gt

## Depuración y generación del archivo de movimientos

### Estructura del archivo (Monetarias y no Monetarias)

|Campo    |Descripción   |
|---    |---   |
|PERIODO    |Período   |
|TIPO_CANAL |Tipo de canal   |
|CANAL  |Canal   |
|TIPO_CLIENTE   |Tipo de cliente   |
|CUITCLIE   |CUIT/CUIL   |
|GRUPO_OPERACION    |Grupo de operación   |
|TIPO_OPERACION |Tipo de operación   |
|TIPO_OPERACION2    |Tipo de operación 2   |
|OPERACION  |Operación   |
|SUCURSAL   |Sucursal   |
|CAB    |CAB   |
|IMPORTE    |Importe   |
|ORIGEN |Origen   |
|TIPO_TRX   |Tipo de transacción   |
|CANAL_RATIO    |**Canal Ratio?**   |
|CLASIF_TRX |Clasificación de la transacción (MONETARIA/NOMONETARIA)   |

In [34]:
uc_tx = [
    "PERIODO",
    "TIPO_CANAL",
    "CANAL",
    "TIPO_CLIENTE",
    "CUITCLIE",
    "GRUPO_OPERACION",
    "TIPO_OPERACION",
    "TIPO_OPERACION2",
    "SUCURSAL",
    "CAB",
    "IMPORTE",
    "ORIGEN",
    "TIPO_TRX",
    "CANAL_RATIO",
    "CLASIF_TRX"
]

t_tx = {
    "PERIODO": str,
    "TIPO_CANAL": str,
    "CANAL": str,
    "TIPO_CLIENTE": str,
    "CUITCLIE": str,
    "GRUPO_OPERACION": str,
    "TIPO_OPERACION": str,
    "TIPO_OPERACION2": str,
    "SUCURSAL": str,
    "CAB": str,
    "IMPORTE": float,
    "ORIGEN": str,
    "TIPO_TRX": str,
    "CANAL_RATIO": str,
    "CLASIF_TRX": str
}

In [48]:
# Lectura de archivo
# Solo tomamos Abril porque en la reunión de Data Driven se concluyó que solo el último es relevante
file = r'\\srvapwb12\SCG_INFORMACION_EXTERNA$\USUARIOS\GCIA BANCA DIGITAL\Detalle_Trx_Monetarias_202204.del'
df_tx = pd.read_csv(file,
    iterator=False,
    nrows=1_000_000,
#    chunksize=1000,
    delimiter='|',
    decimal=",",
    encoding='latin-1',
    dtype=t_tx,
    usecols=uc_tx)

In [36]:
#df_tx = pd.concat([chunk[chunk["CUITCLIE"] != "0"] for chunk in iter_tx])

In [49]:
# Borro movimientos no asociados a ningún cliente
df_tx = df_tx[(df_tx["CUITCLIE"] != "0")]

In [50]:
# Anonimizo la identificación del usuario tomando el hash de la tabla de usuarios
df_tx = pd.merge(df_us, df_tx, how='inner', left_on='DOCUMENTO', right_on='CUITCLIE')
df_tx.drop(["DOCUMENTO", "CUITCLIE"], axis=1, inplace=True)


In [51]:
# Intercambio las columnas PERIODO y ID_USER
cols_tx = list(df_tx.columns)
col_a, col_b = cols_tx.index("ID_USER"), cols_tx.index("PERIODO")
cols_tx[col_b], cols_tx[col_a] = cols_tx[col_a], cols_tx[col_b]
df_tx = df_tx[cols_tx]

In [52]:
df_tx

,PERIODO,ID_USER,TIPO_CANAL,CANAL,TIPO_CLIENTE,GRUPO_OPERACION,TIPO_OPERACION,TIPO_OPERACION2,SUCURSAL,CAB,IMPORTE,ORIGEN,TIPO_TRX,CANAL_RATIO,CLASIF_TRX
0,202204,28c1c77d8e744f38a22406135ca99075,INTERNET,BANCON_MOBILE,EMPRESA,TESORERIA,RECAUDACIONES,RESTO,0,0,5861.26,OMNI,MONETARIAS,DIGITAL,PAGOS
1,202204,6b0dead1f8b848b895536d7031a3ce54,INTERNET,BANCON_WEB,EMPRESA,ALTAS Y GEST COMERCIALES,ALTAS,PLAZO FIJO,0,0,500000.00,OMNI,MONETARIAS,DIGITAL,ALTAS Y GEST COM
2,202204,b6bf42bddaee4d849db1674cf23c47f0,INTERNET,BANCON_WEB,EMPRESA,TESORERIA,DEPOSITOS,ECHEQUE,0,0,50000.00,OMNI,MONETARIAS,DIGITAL,DEPOSITOS
3,202204,b6bf42bddaee4d849db1674cf23c47f0,INTERNET,BANCON_WEB,EMPRESA,TESORERIA,DEPOSITOS,ECHEQUE,0,0,50000.00,OMNI,MONETARIAS,DIGITAL,DEPOSITOS
4,202204,72369e33a8cd4513a4da27f38b959c83,INTERNET,BANCON_WEB,EMPRESA,TESORERIA,DEPOSITOS,ECHEQUE,0,0,209247.30,OMNI,MONETARIAS,DIGITAL,DEPOSITOS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
876280,202204,96d6a65b3a204431862d3d34c5436722,INTERNET,BANCON_MOBILE,INDIVIDUO,TESORERIA,PAGOS,NaN,0,0,25949.00,OMNI,MONETARIAS,DIGITAL,EXCLUIDAS
876281,202204,96d6a65b3a204431862d3d34c5436722,INTERNET,BANCON_MOBILE,INDIVIDUO,TESORERIA,RECAUDACIONES,RESTO,0,0,4344.40,OMNI,MONETARIAS,DIGITAL,PAGOS
876282,202204,87a3b59fbfce494aace64e061c3ef7c0,INTERNET,BANCON_MOBILE,INDIVIDUO,TESORERIA,RECAUDACIONES,RESTO,0,0,4114.00,OMNI,MONETARIAS,DIGITAL,PAGOS
876283,202204,f6a35a687ae4478ab2b0e67ee21317ef,INTERNET,BANCON_MOBILE,INDIVIDUO,TESORERIA,DEPOSITOS,ECHEQUE,0,0,21910.68,OMNI,MONETARIAS,DIGITAL,DEPOSITOS
